In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as apps
import pandas as pd
import kagglehub
from pathlib import Path
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split
import gc


2025-04-08 16:56:28.141593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744127788.153463   69964 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744127788.157046   69964 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744127788.167706   69964 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744127788.167729   69964 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744127788.167730   69964 computation_placer.cc:177] computation placer alr

In [2]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

set_memory_limit(4096)
#set_memory_growth()
keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

1 Physical GPU, 1 Logical GPUs


I0000 00:00:1744127791.687336   69964 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
SEED = 42
BATCH_SIZE = 8
PATIENCE = 5

saved_models_dir = Path("../saved_models")
saved_models_dir.mkdir(parents=True, exist_ok=True)
histories_dir = Path("../histories")
histories_dir.mkdir(parents=True, exist_ok=True)

# Create Datasets

In [4]:
# Download latest version of data
## Use the next 2 lines if not downloaded before
# image_dir = kagglehub.dataset_download("bloox2/fieldplant")
# image_dir = Path(image_dir) / "train"

# My data has been cached
image_dir = "/home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train"
print("Path to dataset files:", image_dir)

Path to dataset files: /home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train


# Create Models

In [5]:
def get_model_and_preprocessing(app_name):
    app = getattr(keras.applications, app_name)
    model_name = dir(app)[0]
    model = getattr(app, model_name)
    input_shape = model().input_shape[1:]
    model = model(include_top=False, input_shape=input_shape)
    model.trainable = False
    preprocessing = getattr(app, "preprocess_input")
    return model, preprocessing

In [6]:
def get_pre_classifier_layers(model_name):
    if "vgg" in model_name:
        return keras.Sequential([
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(1024, activation="relu"),
            keras.layers.Dense(1024, activation="relu") ])
    else:
        return keras.Sequential([keras.layers.GlobalAveragePooling2D()])

In [7]:
def build_model(app_name, activation, num_classes):
    model, preprocessing = get_model_and_preprocessing(app_name)
    pre_classifier_layers = get_pre_classifier_layers(model.name)

    inputs = keras.Input(shape=model.input_shape[1:])
    x = preprocessing(inputs)
    x = model(x, training=False)
    x = pre_classifier_layers(x)
    outputs = keras.layers.Dense(num_classes, activation=activation, name="classifier_layer")(x)

    model_name = model.name
    model = keras.Model(inputs, outputs, name=model_name)

    return model

In [8]:
def get_hyperparameters(methodology):
    multilabel = methodology == "multilabel"
    
    methodologies = ["multiclass", "multilabel"]
    losses = ["categorical_crossentropy", "binary_crossentropy"]
    activation = ["softmax", "sigmoid"]
    metrics = ["categorical_accuracy", "binary_accuracy"]

    idx = methodologies.index(methodology)

    metrics = [metrics[idx]]
    f1_score_weighted = keras.metrics.F1Score(average="weighted", threshold=0.5 if multilabel else None, name="f1_score_weighted", dtype=None)
    f1_score_per_class = keras.metrics.F1Score(average=None, threshold=0.5 if multilabel else None, name="f1_score_per_class", dtype=None)
    metrics.append(f1_score_weighted)
    metrics.append(f1_score_per_class)
        
    hyperparams = [losses[idx], activation[idx], metrics]
    
    return hyperparams

In [9]:
def print_model_info(model):
    print("print_model_info() start")
    compile_config = model._compile_config.config
    optimizer = compile_config['optimizer'].get_config()
    classifier_activation = model.get_layer(name="classifier_layer").activation.__name__

    print("Model name:", model.name)
    print("Input shape:", model.input_shape)
    print("Optimizer name:", optimizer['name'], "learning_rate:", np.round(optimizer['learning_rate'], 6))
    print("Loss:", compile_config['loss'])
    print("Metrics:")
    for metric in compile_config['metrics']:
        print(metric if isinstance(metric, str) else metric.get_config())
    print("Classifier layer activation function:", classifier_activation)
    print()
    print(model.summary(expand_nested=True, show_trainable=True))
    print()

    print("print_model_info() end")
    print()


In [10]:
def get_callbacks(model_name, fine_tuning=False):
    cbs = [
        keras.callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights=True, baseline=None, verbose=1),
        keras.callbacks.ModelCheckpoint(filepath=f"{saved_models_dir}/{model_name}.keras", save_best_only=True, monitor="val_loss", verbose=1, initial_value_threshold=None)]
    return cbs


# GET DATASETS

In [11]:
def get_dataframe(filtered=False, sample=False):
    filename = "filtered" if filtered else "unfiltered"
    all_csv_files = list(Path("../data").glob("*"))
    csv_file = [csv for csv in all_csv_files if filename in csv.name][0]
    df = pd.read_csv(csv_file)
    print()

    if sample:
        print("Using sampled DF")
        df = df.sample(frac=0.2)
        df = df.loc[(df!=0).any(axis=1)]
    print("df shape:", df.shape)
    return df   

In [12]:
def get_train_test_splits(df, filtered=False, test_size=0.2):
    col_names = list(df.columns)
    split_fn = get_stratified_splits if filtered else get_nonstratified_splits
    (X_train, X_test, X_val, y_train, y_test, y_val) = split_fn(df, test_size=test_size)
    train_df = pd.merge(X_train, y_train, left_index=True, right_index=True)
    test_df = pd.merge(X_test, y_test, left_index=True, right_index=True)
    val_df = pd.merge(X_val, y_val, left_index=True, right_index=True)
    
    train_df.columns = col_names
    test_df.columns = col_names
    val_df.columns = col_names

    return train_df, test_df, val_df
    
def get_stratified_splits(df, test_size=0.2):
    columns = list(df.columns)
    X = df.filename.to_frame().to_numpy()
    y = df.drop(columns=["filename"]).to_numpy()

    X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
    X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)
    datasets = (X_train, X_test, X_val, y_train, y_test, y_val)
    datasets = [pd.DataFrame(dataset) for dataset in datasets]
    return tuple(datasets)

def get_nonstratified_splits(df, test_size=0.2):
    X = df.filename
    y = df.drop(columns=["filename"])
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=SEED)

    return (X_train, X_test, X_val, y_train, y_test, y_val)

In [13]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + "/" + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img
    
def process_dataset(filename, labels, img_size):
    img = decode_img(filename, img_size=img_size)
    return img, labels

def configure_datasets_for_performance(datasets, shuffle=False, batch_size=BATCH_SIZE):
    configured_datasets = []
    for dataset in datasets:
        if shuffle:
            dataset = dataset.shuffle(buffer_size=dataset.cardinality(), reshuffle_each_iteration=True)
        dataset = dataset.batch(batch_size=batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.cache()
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        configured_datasets.append(dataset)
    return tuple(configured_datasets)

In [14]:
def datasets_from_dataframes(img_size, splits=None):
    datasets = []
    for split in splits:
        img = split.filename
        labels = split.drop(columns=["filename"])
        dataset = tf.data.Dataset.from_tensor_slices((img, labels))
        dataset = dataset.map(lambda x,y: process_dataset(x,y, img_size))
        datasets.append(dataset)
    return tuple(datasets)

In [15]:
def get_datasets(img_size, df, test_size=0.2):
    splits = get_train_test_splits(df, test_size=test_size)
    datasets = datasets_from_dataframes(img_size, splits=splits)
    for dataset in zip(["train", "val", "test"], datasets):
        print(dataset[0],"size:", len(dataset[1]))
    return datasets

# CREATE MODELS

In [16]:
def freeze_model(model):
    print()
    print("freeze_model() start")

    base_layer_name = ""
    num = 0
    if "mobilenet" in model.name:
        base_layer_name = "mobilenetv2_1.00_224"
        num = 120
    elif "vgg16" in model.name:
        base_layer_name = "vgg16"
        num = 14
    elif "inception_v3" in model.name:
        base_layer.name = "inception_v3"
        num = 172
    elif "resnet" in model.name:
        base_layer.name = "inception_resnet_v2"
        num = 516

    print(f"Freeze layers until layer {num}")
    for layer in model.get_layer(name=base_layer_name).layers[:num]:
        layer.trainable = False
    for layer in model.get_layer(name=base_layer_name).layers[num:]:
        layer.trainable = True

    print("freeze_model() end")
    print()

In [17]:
def compile_model(model, loss=None, metrics=[], fine_tuning=False):
    print()
    print("compile_model() start")
    opt = keras.optimizers.Adam
    lr = (float(opt().learning_rate) / 10) if fine_tuning else float(opt().learning_rate)
    
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(learning_rate=lr, gradient_accumulation_steps=4),
        metrics=metrics
    )
    print("compile_model() end")
    print()

In [18]:
def train_model(model=None, train_ds=None, val_ds=None, epochs=100, loss=None, metrics=[], fine_tuning=False, initial_epoch=0):
    print()
    print("train_model() start")

    string_suffix = "FT" if fine_tuning else "CLF"
    print("Training -", string_suffix)

    if fine_tuning:
        freeze_model(model)
        
    compile_model(model, loss=loss, metrics=metrics, fine_tuning=fine_tuning)
    print_model_info(model)
       
    cbs = get_callbacks(model.name, fine_tuning=fine_tuning)

    print(f"Starting training at epoch {initial_epoch+1}." )
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs, initial_epoch=initial_epoch)
    history_df = pd.DataFrame(history.history)
    history_df['model'] = model.name
    history_df['epoch'] = history.epoch
    history_df['type'] = string_suffix
    display(history_df)
    history_df.to_csv(str(histories_dir) + "/" + model.name + string_suffix + ".csv", index=False)  
    
    best_epoch = history_df.loc[history_df.val_loss == history_df.val_loss.min(), "epoch"].values[0] + 1
    print(f"Best epoch number: {best_epoch}")

    print("train_model() end")
    print()

    return best_epoch
    

In [19]:
def run_all(epochs=5, sample=True):
    print("run all() start")

    i = 0
    num_models = 0

    # Use for training the entire loop
    app_names = ["mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"]
    # methodologies = ["multiclass", "multilabel"]
    # filter_options = [False, True]

    # # Use for smaller combinations
    #app_names = ["mobilenet_v2"]
    methodologies = ["multiclass", "multilabel"]
    filter_options = [False, True]


    # Get total count of models to be training
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
            for app_name in app_names:
                num_models += 1

    # Training Loop
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
                
            loss, activation, metrics = get_hyperparameters(methodology)
    
            df = get_dataframe(filtered=filtered, sample=sample)
            num_classes = len(df.columns[1:])
            
            for app_name in app_names:
                i += 1
                print()
                print(f"Model: {i} of {num_models}")
                print("Filtered dataset:", filtered)
                print("Methodology:", methodology)
       
                model = build_model(app_name, activation=activation, num_classes=num_classes)

                img_size = model.input_shape[1:3]
                
                datasets = get_datasets(img_size, df=df)
                train_ds, test_ds, val_ds = configure_datasets_for_performance(datasets)

                model.name = model.name + "_" + ("filtered" if filtered else "unfiltered") + "_" + methodology

                # Train top Classifier
                best_epoch = train_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=False, initial_epoch=0)
                # Fine Tuning
                ft_start_epoch = best_epoch
                train_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=True, initial_epoch=ft_start_epoch)

                # Clear memory
                del model
                gc.collect()
                tf.keras.backend.clear_session()
    print("run all() end")
    print()



In [20]:
model = run_all(epochs=100, sample=False)

run all() start

df shape: (5156, 28)

Model: 1 of 3
Filtered dataset: False
Methodology: multiclass
train size: 4124
val size: 516
test size: 516

train_model() start
Training - CLF

compile_model() start
compile_model() end

print_model_info() start
Model name: mobilenetv2_1.00_224_unfiltered_multiclass
Input shape: (None, 224, 224, 3)
Optimizer name: adam learning_rate: 0.001
Loss: categorical_crossentropy
Metrics:
categorical_accuracy
{'name': 'f1_score_weighted', 'dtype': 'float32', 'average': 'weighted', 'threshold': None}
{'name': 'f1_score_per_class', 'dtype': 'float32', 'average': None, 'threshold': None}
Classifier layer activation function: softmax



Model: "mobilenetv2_1.00_224_unfiltered_multiclass"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_2 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ true_divide (TrueDivide)    │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ subtract (Subtract)         │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenetv2_1.00_224        │ (None, 7, 7, 1280)    │  2,257,984 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ input_layer_1          │ (None, 224, 224, 3)   │          0 │   -   │
│ (InputLayer)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ Conv1 (Conv2D)         │ (None, 112, 112, 32)  │        864 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn_Conv1               │ (None, 112, 112, 32)  │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ Conv1_relu (ReLU)      │ (None, 112, 112, 32)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 32)  │        288 │   N   │
│ expanded_conv_depthwise     │                       │            │       │
│ (DepthwiseConv2D)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 32)  │        128 │   N   │
│ expanded_conv_depthwise_BN  │                       │            │       │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 32)  │          0 │   -   │
│ expanded_conv_depthwise_re… │                       │            │       │
│ (ReLU)                      │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ expanded_conv_project  │ (None, 112, 112, 16)  │        512 │   N   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 16)  │         64 │   N   │
│ expanded_conv_project_BN    │                       │            │       │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_expand         │ (None, 112, 112, 96)  │      1,536 │   N   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_expand_BN      │ (None, 112, 112, 96)  │        384 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_expand_relu    │ (None, 112, 112, 96)  │          0 │   -   │
│ (ReLU)                      │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_pad            │ (None, 113, 113, 96)  │          0 │   -   │
│ (ZeroPadding2D)             │                       │            │     

 Total params: 2,292,571 (8.75 MB)

 Trainable params: 34,587 (135.11 KB)

...

I0000 00:00:1744127797.776188   70073 service.cc:152] XLA service 0x7fe6a8053270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744127797.776240   70073 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-08 16:56:38.376507: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744127798.998452   70073 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/516 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - categorical_accuracy: 0.0683 - f1_score_per_class: 0.0137 - f1_score_weighted: 0.0614 - loss: 3.4541

I0000 00:00:1744127805.395594   70073 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 24/516 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - categorical_accuracy: 0.0822 - f1_score_per_class: 0.0196 - f1_score_weighted: 0.0671 - loss: 3.4190

KeyboardInterrupt: 